In [1]:
from utils import (
    extract_nmf_features,
    extract_nmf,
    tweet_level_feature_generation,
    split_all_data,
    nn_tweet_only,
    add_nn_signal,
    add_feat,
    calculate_similarity,
    tweet_level_features,
    generate_sentiment_feature
)

import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
# import nltk
# nltk.download('vader_lexicon')

In [10]:
random_seed = 3850

In [3]:
data_dict = split_all_data(.8, dev_test_split=.5, random_state=random_seed)
train_df = data_dict['train']
dev_df = data_dict['dev']
test_df = data_dict['test']
print('shape of train_df: ', train_df.shape)
print('shape of dev_df  : ', dev_df.shape)
print('shape of test_df : ', test_df.shape)

Loading the data...
Splitting the data...
shape of train_df:  (9460, 6)
shape of dev_df  :  (1183, 6)
shape of test_df :  (1183, 6)


In [4]:
train_df.sample(5, random_state=random_seed)

,ID,profile,tweet,neighbor,domain,label
5556,32447721,"{'id': '32447721 ', 'id_str': '32447721 ', 'na...",[I look forward to not being able to find a Nv...,"{'following': ['818668163466457088', '11995844...",[Entertainment],0
814,3017957005,"{'id': '3017957005 ', 'id_str': '3017957005 ',...",[RT @KendrickPerkins: Ladies and Gentlemen we ...,"{'following': ['63515922', '71415087', '293008...",[Politics],1
10475,264962685,"{'id': '264962685 ', 'id_str': '264962685 ', '...",[Nothing new comes from a standard response. #...,"{'following': ['760210372888563712', '86116147...",[Sports],1
7336,1074071841885417472,"{'id': '1074071841885417472 ', 'id_str': '1074...",[RT @DocSoph: Looking for unique &amp; innovat...,"{'following': ['1094029591054872576', '2288915...",[Sports],1
4999,1239433466460155904,"{'id': '1239433466460155904 ', 'id_str': '1239...",[RT @Sunflower_Films: Retweet this if you've l...,"{'following': ['1218302853502177280', '2425571...",[Entertainment],1


In [5]:
vectorizer, nn_model = nn_tweet_only(train_df, dev_df)

Prepping tweet data...
Vectorizing tweets...
Epoch 1/20


c:\ProgramData\Anaconda3\envs\ML_w_tf_env\lib\site-packages\keras\backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


266/266 [==============================] - 12s 39ms/step - loss: 0.5717 - accuracy: 0.7031 - val_loss: 0.5448 - val_accuracy: 0.7262
Epoch 2/20
266/266 [==============================] - 1s 5ms/step - loss: 0.4599 - accuracy: 0.7872 - val_loss: 0.5422 - val_accuracy: 0.7206
Epoch 3/20
266/266 [==============================] - 1s 5ms/step - loss: 0.3735 - accuracy: 0.8430 - val_loss: 0.5797 - val_accuracy: 0.7206
Epoch 4/20
266/266 [==============================] - 1s 5ms/step - loss: 0.2600 - accuracy: 0.9031 - val_loss: 0.6917 - val_accuracy: 0.7003
Restoring model weights from the end of the best epoch.
Epoch 00004: early stopping


In [6]:
train_df = add_nn_signal(train_df, vectorizer, nn_model)
test_df = add_nn_signal(test_df, vectorizer, nn_model)

In [7]:
train_df_stage_1 = add_feat(train_df)
test_df_stage_1 = add_feat(test_df)

In [8]:
train_df_stage_1.sample(5, random_state=random_seed)

,ID,profile,tweet,neighbor,domain,label,nn_signal,lang_count,tweet_min_len,tweet_max_len,tweet_av_len,tweet_len_std,user_name_len,screen_name_len,user_name_chars,protected,verified,has_url,followers_count,friends_count,favourites_count,days_old
4794,353345238,"{'id': '353345238 ', 'id_str': '353345238 ', '...",[“Turntablism”Ft @Djksity Available Here 👇🏽\nh...,"{'following': ['1049996402', '13749282', '9197...",[Politics],1,0.632723,2,13,315,191.301020,85.951732,6,8,6,0,0,1,1475,2898,721,3915
4495,251652744,"{'id': '251652744 ', 'id_str': '251652744 ', '...",[RT @PeterAdeniyi7: Looking for a game this co...,"{'following': ['2519538916', '461227649', '969...",[Politics],1,0.864660,3,9,185,94.402010,46.934459,12,9,10,0,0,0,549,375,62,4094
910,797685578,"{'id': '797685578 ', 'id_str': '797685578 ', '...",[#M25 anti-clockwise between J25 (Cheshunt / E...,"{'following': ['233611355'], 'follower': ['113...",[Sports],1,0.637529,1,105,184,151.505000,20.945166,19,12,14,0,0,1,45436,1,0,3527
8790,16334281,"{'id': '16334281 ', 'id_str': '16334281 ', 'na...",[Miami-Dade's main jail turned into blood dona...,None,"[Politics, Entertainment]",0,0.178129,1,48,309,164.465000,63.008720,11,9,9,0,1,1,104451,1430,4721,4973
2947,987666672684183552,"{'id': '987666672684183553 ', 'id_str': '98766...","[Nakakagago lang!\n, @_SMFFFF sila ian stem b ...","{'following': ['1160498466306449408', '1072092...",[Politics],0,0.308125,5,5,190,42.707071,27.038227,14,10,9,0,0,0,6431,6290,1615,1470


In [9]:
train_df_stage_1 = tweet_level_features(train_df_stage_1)
test_df_stage_1 = tweet_level_features(test_df_stage_1)

In [10]:
train_df_stage_1 = generate_sentiment_feature(train_df_stage_1)
test_df_stage_1 = generate_sentiment_feature(test_df_stage_1)

In [11]:
train_df_stage_1.loc[:, 'similarity'] = calculate_similarity(train_df_stage_1['tweet'])
test_df_stage_1.loc[:, 'similarity'] = calculate_similarity(test_df_stage_1['tweet'])

In [12]:
train_df_stage_1 = tweet_level_feature_generation(train_df_stage_1, normalize=True)
test_df_stage_1 = tweet_level_feature_generation(test_df_stage_1, normalize=True)

In [13]:
train_df_stage_1.sample(5, random_state=random_seed)

,ID,profile,tweet,neighbor,domain,label,nn_signal,lang_count,tweet_min_len,tweet_max_len,tweet_av_len,tweet_len_std,user_name_len,screen_name_len,user_name_chars,protected,verified,has_url,followers_count,friends_count,favourites_count,days_old,listed_count,statuses_count,description_len,sentiment,similarity,num_mentions,num_links,retweet,num_tweets
4794,353345238,"{'id': '353345238 ', 'id_str': '353345238 ', '...",[“Turntablism”Ft @Djksity Available Here 👇🏽\nh...,"{'following': ['1049996402', '13749282', '9197...",[Politics],1,0.632723,2,13,315,191.301020,85.951732,6,8,6,0,0,1,1475,2898,721,3915,16,1263,112,Positive,0.605875,2.938776,1.086735,0.209184,196
4495,251652744,"{'id': '251652744 ', 'id_str': '251652744 ', '...",[RT @PeterAdeniyi7: Looking for a game this co...,"{'following': ['2519538916', '461227649', '969...",[Politics],1,0.864660,3,9,185,94.402010,46.934459,12,9,10,0,0,0,549,375,62,4094,3,2180,133,Positive,0.656970,1.512563,0.281407,0.628141,199
910,797685578,"{'id': '797685578 ', 'id_str': '797685578 ', '...",[#M25 anti-clockwise between J25 (Cheshunt / E...,"{'following': ['233611355'], 'follower': ['113...",[Sports],1,0.637529,1,105,184,151.505000,20.945166,19,12,14,0,0,1,45436,1,0,3527,145,97236,135,Negative,0.940769,0.000000,1.000000,0.000000,200
8790,16334281,"{'id': '16334281 ', 'id_str': '16334281 ', 'na...",[Miami-Dade's main jail turned into blood dona...,None,"[Politics, Entertainment]",0,0.178129,1,48,309,164.465000,63.008720,11,9,9,0,1,1,104451,1430,4721,4973,2021,199448,51,Negative,0.767780,0.955000,1.005000,0.240000,200
2947,987666672684183552,"{'id': '987666672684183553 ', 'id_str': '98766...","[Nakakagago lang!\n, @_SMFFFF sila ian stem b ...","{'following': ['1160498466306449408', '1072092...",[Politics],0,0.308125,5,5,190,42.707071,27.038227,14,10,9,0,0,0,6431,6290,1615,1470,1,2514,6,Positive,0.433443,0.282828,0.328283,0.040404,198


In [ ]:
# save point
train_df_stage_1.to_json('Twibot-20/data_creation/train_df_stage_1.json')
test_df_stage_1.to_json('Twibot-20/data_creation/test_df_stage_1.json')

In [4]:
train_df_stage_1 = pd.read_json('Twibot-20/data_creation/train_df_stage_1.json')
test_df_stage_1 = pd.read_json('Twibot-20/data_creation/test_df_stage_1.json')

In [5]:
train_df_stage_2 = extract_nmf(train_df_stage_1)
test_df_stage_2 = extract_nmf(test_df_stage_1)

In [6]:
train_df_stage_2.to_json('Twibot-20/data_creation/train_df_stage_2.json')
test_df_stage_2.to_json('Twibot-20/data_creation/test_df_stage_2.json')

In [7]:
train_df_stage_2 = pd.read_json('Twibot-20/data_creation/train_df_stage_2.json')
test_df_stage_2 = pd.read_json('Twibot-20/data_creation/test_df_stage_2.json')

In [8]:
train_df_stage_2 = extract_nmf_features(train_df_stage_2)
test_df_stage_2 = extract_nmf_features(test_df_stage_2)

In [11]:
train_df_stage_2.sample(5, random_state=random_seed)

,ID,profile,tweet,neighbor,domain,label,nn_signal,lang_count,tweet_min_len,tweet_max_len,tweet_av_len,tweet_len_std,user_name_len,screen_name_len,user_name_chars,protected,verified,has_url,followers_count,friends_count,favourites_count,days_old,listed_count,statuses_count,description_len,sentiment,similarity,num_mentions,num_links,retweet,num_tweets,topic_dist,topic_skew,topic_kurtosis,topic_gini,topic_std
4794,353345238,"{'id': '353345238 ', 'id_str': '353345238 ', '...",[“Turntablism”Ft @Djksity Available Here 👇🏽\nh...,"{'following': ['1049996402', '13749282', '9197...",[Politics],1,0.632723,2,13,315,191.301020,85.951732,6,8,6,0,0,1,1475,2898,721,3915,16,1263,112,Positive,0.605875,2.938776,1.086735,0.209184,196,"[0.6275510204, 0.3928571429, 0.561224489800000...",-0.000065,-1.595921,0.432486,0.255895
4495,251652744,"{'id': '251652744 ', 'id_str': '251652744 ', '...",[RT @PeterAdeniyi7: Looking for a game this co...,"{'following': ['2519538916', '461227649', '969...",[Politics],1,0.864660,3,9,185,94.402010,46.934459,12,9,10,0,0,0,549,375,62,4094,3,2180,133,Positive,0.656970,1.512563,0.281407,0.628141,199,"[0.6884422111, 0.8040201005000001, 0.0, 0.3316...",0.269811,-1.435468,0.489406,0.281941
910,797685578,"{'id': '797685578 ', 'id_str': '797685578 ', '...",[#M25 anti-clockwise between J25 (Cheshunt / E...,"{'following': ['233611355'], 'follower': ['113...",[Sports],1,0.637529,1,105,184,151.505000,20.945166,19,12,14,0,0,1,45436,1,0,3527,145,97236,135,Negative,0.940769,0.000000,1.000000,0.000000,200,"[0.465, 0.045, 0.43, 0.43, 0.005, 0.465, 0.425...",-0.230869,-1.476152,0.349286,0.174388
8790,16334281,"{'id': '16334281 ', 'id_str': '16334281 ', 'na...",[Miami-Dade's main jail turned into blood dona...,None,"[Politics, Entertainment]",0,0.178129,1,48,309,164.465000,63.008720,11,9,9,0,1,1,104451,1430,4721,4973,2021,199448,51,Negative,0.767780,0.955000,1.005000,0.240000,200,"[0.77, 0.735, 0.5750000000000001, 0.08, 0.775,...",-0.056507,-1.376145,0.356653,0.286312
2947,987666672684183552,"{'id': '987666672684183553 ', 'id_str': '98766...","[Nakakagago lang!\n, @_SMFFFF sila ian stem b ...","{'following': ['1160498466306449408', '1072092...",[Politics],0,0.308125,5,5,190,42.707071,27.038227,14,10,9,0,0,0,6431,6290,1615,1470,1,2514,6,Positive,0.433443,0.282828,0.328283,0.040404,198,"[0.0737704918, 0.3852459016, 0.868852459000000...",0.608577,-1.097676,0.560014,0.320677


In [12]:
# save point
train_df_stage_2.to_json('Twibot-20/data_creation/train_final_dataset.json')
test_df_stage_2.to_json('Twibot-20/data_creation/test_final_dataset.json')